In [ ]:
import numpy as np
import tensorflow as tf
import sklearn

# Data Processing

### Import Data

In [ ]:
raw_data = np.genfromtxt('x09.txt', skip_header= 36)
print(f'Number of Rows: {len(raw_data)}')
print(raw_data[:5])

### Visualization

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

xs = np.array(raw_data[: , 2] , dtype=np.float32) # 2nd Column
ys = np.array(raw_data[: , 3] , dtype=np.float32) # 3rd Column
zs = np.array(raw_data[: , 4] , dtype=np.float32) # 4th Column

fig, axs = plt.subplots(1,2,figsize = (7,5),subplot_kw={'projection': '3d'})

axs[0].scatter(xs, ys, zs)
axs[0].view_init(5,15)
axs[0].set_xlabel('Weight')
axs[0].set_ylabel('Age')
axs[0].set_zlabel('Blood fat')

axs[1].scatter(xs, ys, zs)
axs[1].view_init(25,45)
axs[1].set_xlabel('Weight')
axs[1].set_ylabel('Age')
axs[1].set_zlabel('Blood fat')


### Seperate Data

In [ ]:
X = np.array(raw_data[:, 2:4], dtype=np.float32) # divide input value
y =  np.array(raw_data[:, 4], dtype=np.float32) #divide output value
y = y.reshape(len(y), 1)

print(f' X values: (multi values - input)\n {X[:5]}')
print(f' y value: (single value - output)\n {y[:5]}')


### Scale to 0~1

In [5]:
import sklearn.preprocessing

scaler = sklearn.preprocessing.MinMaxScaler(feature_range= (0,1))
scaled_X = scaler.fit_transform(X)

### Create Model

In [ ]:
model1 = tf.keras.models.Sequential([ # maually set hidden layer parameters
    
    tf.keras.layers.Dense(units=1, input_shape=(2,), activation= 'relu') # 1st Hidden Layer gets 2 input (Age, Weight)

])

model1.compile(
    optimizer= 'rmsprop', # ex)"rnsprop" 
    loss = "mse", # ex)"mse" method for varify the prediction with actual resul
)

model2 = tf.keras.models.Sequential([ # maually set hidden layer parameters
    
    tf.keras.layers.Dense(units=2, input_shape=(2,), activation= 'relu'), # 1st Hidden Layer gets 2 input (Age, Weight)
    tf.keras.layers.Dense(units=2, activation= 'softmax')
])

model2.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001), # ex)"rnsprop" 
    loss = "sparse_categorical_crossentropy" # ex)"mse" method for varify the prediction with actual resul
)


model3 = tf.keras.models.Sequential([ # maually set hidden layer parameters
    
    tf.keras.layers.Dense(units=2, input_shape=(2,), activation= 'relu'), # 1st Hidden Layer gets 2 input (Age, Weight)
    tf.keras.layers.Dense(units=2, activation= 'relu'),
    tf.keras.layers.Dense(units=2, activation= 'softmax')
])

model3.compile(
    optimizer= 'rmsprop', # ex)"rnsprop" 
    loss = "mse" # ex)"mse" method for varify the prediction with actual resul
)


# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # ex)"rnsprop" 
#     loss = "sparse_categorical_crossentropy", # ex)"mse" method for varify the prediction with actual resul
# )

In [ ]:
class Callback(tf.keras.callbacks.Callback):
    SHOW_NUMBER = 10
    counter = 0
    epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch

    def on_train_batch_end(self, batch, logs=None):
        if self.counter == self.SHOW_NUMBER or self.epoch == 1:
            print('Epoch: ' + str(self.epoch) + ' loss: ' + str(logs['loss']))
            if self.epoch > 1:
                self.counter = 0
        self.counter += 1

hist1 = model1.fit(
    X,
    y,
    batch_size = 1,
    epochs= 5000, 
    callbacks=[Callback()],
    verbose=0
)

hist3 = model3.fit(
    X,
    y,
    batch_size = 1,
    epochs= 5000, 
    callbacks=[Callback()],
    verbose=0
)

In [ ]:
fig, axs = plt.subplots(2,1 , figsize=(7,3))
axs[0].plot(hist1.history['loss'])
# axs[1].plot(hist2.history['loss'])
axs[1].plot(hist3.history['loss'])

In [ ]:
# model1.predict(np.array([100, 44]).reshape(1,2))
# model2.predict(np.array([100, 44]).reshape(1,2))

weight = 100
age = 44

print(model3.predict(np.array([weight, age]).reshape(1,2)))
